In [ ]:
#get a list of stop words to remove unnecessary words from my counts
import nltk
from nltk.corpus import stopwords
stopit=set(stopwords.words('english'))

In [ ]:
#create a dictionary of positive words and positive reviews

from collections import Counter, defaultdict

postext=[]
poswords=[]
positiveword=[]
posword=[]

with open ('/Users/daniaalma/Desktop/hotelPosT-train.txt','r') as posfile:
    for line in posfile.readlines():
        identifier, posreview = line.strip().split("\t")
        postext.append(posreview)
        positiveword=" ".join(postext)

poswords=positiveword.split(" ") 
#print(poswords)

In [ ]:
#create a dictionary of negative words and negative reviews

negtext=[]
negwords=[]
negativeword=[]
negword=[]

with open ('/Users/daniaalma/Desktop/hotelNegT-train.txt','r') as negfile:
    for line in negfile.readlines():
        identifier, negreview = line.strip().split("\t")
        negtext.append(negreview)
        negativeword=" ".join(negtext)

negwords=negativeword.split(" ")
#print(negwords)

In [ ]:
#make a count of how many reviews there are total(pos and neg)

numofreviewstotal=postext+negtext

#print(len(numofreviewstotal))

In [ ]:
#create a dictionary of all of the words that occured in all of the reviews....will this affect the algorithm?

allwords=negwords+poswords

allreviews_words_dictionary=set(allwords)
allreviews_words_dictionary.remove('')

#print(allreviews_words_dictionary)

In [ ]:
#get a list of all the words that occur in both the pos and negative and remove them....maybe i should get max counts
#and use that instead? i noticed that friendly was used in both and i'm not sure if that would help...lets see

commonwords=[]
#lets try to make a dictionary of the overlapping words in both postext and negtext
for commonword in allwords:
    if commonword in poswords:
        if commonword in negwords:
            commonwords.append(commonword)
            
#print(commonwords)

In [ ]:
#make a new word dictionary for the pos and neg that are not common with each other

for pieces in poswords:
    if pieces not in commonwords:
        if pieces not in stopit:
            posword.append(pieces)
    else:
        continue

for piece in negwords:
    if piece not in commonwords:
        if piece not in stopit:
            negword.append(piece)
    else:
        continue


In [ ]:
#LOGPRIORNEGATIVE AND LOGPRIORPOSITIVE

#start algorithm for logprior
from math import log

logpriorpositive=log(len(postext)/len(numofreviewstotal))

logpriornegative=log(len(negtext)/len(numofreviewstotal))
#these are my log priors

print(logpriornegative)

In [ ]:
#get the counts of the pos and neg words (not common)
#oh man i think this might affect my algorithm....... 

poswordcounts = defaultdict(Counter)
poswordcounts=Counter(posword)


negwordcounts = defaultdict(Counter)
negwordcounts=Counter(negword)

#print(negwordcounts)


In [ ]:
#LOGLIKELIHOOD

loglikelihood={}

for word,count in poswordcounts.items():
    loglikelihood[word]={'NEG':0}
    loglikelihood[word]['POS']=log((count+1)/(len(allwords)+len(set(allwords))))
    
    
for word,count in negwordcounts.items():
    if word in loglikelihood:
        loglikelihood[word]['NEG']=log((count+1)/(len(allwords)+len(set(allwords))))
    else:
        loglikelihood[word]={'POS':0}
        loglikelihood[word]['NEG']=log((count+1)/(len(allwords)+len(set(allwords))))
#print(loglikelihood)

In [ ]:
#algorithm

def testnaivebayes(text,logpriornegative,logpriorpositive,loglikelihood,V):
    #one review at a time, pos or neg?
    negative_predict=0
    positive_predict=0
    for word in text:
        if word in loglikelihood:
            negative_predict+=logpriornegative+loglikelihood[word]['NEG']
            positive_predict+=logpriorpositive+loglikelihood[word]['POS']
        else:
            negative_predict+=logpriornegative
            positive_predict+=logpriorpositive
    if negative_predict<positive_predict:
        #log so smaller is better
        return 'NEG'
    else:
        return 'POS'
    #still max 


In [ ]:
#run my test data and create an output

testresultsdict={}
with open('/Users/daniaalma/Desktop/testset.txt', 'r') as testset:
    for line in testset:
        testidentifier,testreview = line.strip().split('\t')
        testreviewclean=testreview
        label=testnaivebayes(testreviewclean.split(" "),logpriornegative,logpriorpositive,loglikelihood,allreviews_words_dictionary)
        testresultsdict[testidentifier]=label
    
with open('/Users/daniaalma/Desktop/Dania-assgn3-test-output.txt', 'a') as output:
    for key,value in testresultsdict.items():
        output.write("{}\t{}".format(key,value))
        output.write("\n")